In [9]:
import sys
sys.path.append("../..")
import numpy as np
import matplotlib.pyplot as plt
from pyfusion import Fusion, AnalogTwo
from scipy.signal import welch
%matplotlib qt

In [2]:
def load_bitstream_to_flash(cmd_if:serial.Serial, data_if:serial.Serial, path:str, partition_index=0):
    print(f"CMD if = {cmd_if.port}  DATA if = {data_if.port}")
    with open(path, 'rb') as bs:
        data=bs.read()#[26:]
        bs_size=len(data)
        cmd = bytearray(5)
        print(f"Writing bitstream to flash, size={bs_size}")
        cmd[0]=4
        cmd[1]=0
        cmd[2]=0
        cmd[3]=0
        cmd[4]=partition_index&255
        print(cmd)
        cmd_if.write(cmd)
        data_if.write(data)
        remaining = int(1024*1024/8 - bs_size)
        print(f"Stuff with {remaining} x 0xff")
        data_if.write(bytearray([255]* remaining ))

NameError: name 'serial' is not defined

In [3]:
analog_two = AnalogTwo(bitstream_path='../../rtl/top/analog_two/build/top.bit')
analog_two.open()

CMD if = /dev/ttyACM2  DATA if = /dev/ttyACM3
Loading bitstream, size=348667
bytearray(b'\x01\x00\x05Q\xfb')


In [32]:
analog_two.get_samples(count=20000)
v=analog_two.get_samples(count=20000000)

In [4]:
v

[array([-3,  0,  1, ...,  0, -4,  3], dtype=int16),
 array([-3,  0,  1, ...,  0, -4,  3], dtype=int16),
 array([ -8,  -7,  -2, ...,  -6,  -3, -10], dtype=int16),
 array([ -8,  -7,  -2, ...,  -6,  -3, -10], dtype=int16)]

In [61]:
analog_two.data_if.flush()
analog_two.data_if.read(2**20)
v=analog_two.data_if.read(2**20)

In [62]:
[hex(i) for i in v[:100]]

['0xfc',
 '0xff',
 '0xfc',
 '0xff',
 '0xf0',
 '0xf',
 '0x8b',
 '0xf8',
 '0xfc',
 '0xff',
 '0xfc',
 '0xff',
 '0xf7',
 '0xff',
 '0xf7',
 '0xff',
 '0xf0',
 '0xf',
 '0x8c',
 '0xf8',
 '0x1',
 '0x0',
 '0x1',
 '0x0',
 '0xff',
 '0xff',
 '0xff',
 '0xff',
 '0xf0',
 '0xf',
 '0x8d',
 '0xf8',
 '0xff',
 '0xff',
 '0xff',
 '0xff',
 '0xf6',
 '0xff',
 '0xf6',
 '0xff',
 '0xf0',
 '0xf',
 '0x8e',
 '0xf8',
 '0xff',
 '0xff',
 '0xff',
 '0xff',
 '0xf9',
 '0xff',
 '0xf9',
 '0xff',
 '0xf0',
 '0xf',
 '0x8f',
 '0xf8',
 '0x3',
 '0x0',
 '0x3',
 '0x0',
 '0xfd',
 '0xff',
 '0xfd',
 '0xff',
 '0xf0',
 '0xf',
 '0x90',
 '0xf8',
 '0xfe',
 '0xff',
 '0xfe',
 '0xff',
 '0xf6',
 '0xff',
 '0xf6',
 '0xff',
 '0xf0',
 '0xf',
 '0x91',
 '0xf8',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0xfe',
 '0xff',
 '0xfe',
 '0xff',
 '0xf0',
 '0xf',
 '0x92',
 '0xf8',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0xfc',
 '0xff',
 '0xfc',
 '0xff']

In [63]:
np.frombuffer(v[4:], dtype=np.int16)[:20]

array([ 4080, -1909,    -4,    -4,    -9,    -9,  4080, -1908,     1,
           1,    -1,    -1,  4080, -1907,    -1,    -1,   -10,   -10,
        4080, -1906], dtype=int16)

In [67]:
plt.figure()
plt.plot(np.frombuffer(v[4:], dtype=np.int16)[5::6])

In [5]:
fig,ax= plt.subplots(4, 1)
for a,c in zip(ax,v):
    a.plot(c)

In [6]:
fig,ax= plt.subplots(4, 1)
for a,c in zip(ax,v):
    fft = np.fft.fft(c-np.mean(c))/len(c)
    freq = np.fft.fftfreq(len(fft), d=1./2e5)[0:len(fft)//2]
    fft = np.abs(fft[0:len(fft)//2])
    a.plot(freq, fft)
    a.semilogx()

In [33]:
fig,ax= plt.subplots(4, 1)
for i in range(4):
    f,p=welch(v[i]*3.2768*2.5/65536.,nperseg=2**22, scaling='density', fs=42e6/2**5/4)
    ax[i].plot(f,np.sqrt(p))
    ax[i].semilogx()

In [8]:
fig,ax= plt.subplots(4, 1)
for i in range(4):
    ax[i].specgram(v[i])


In [627]:
d=serial.Serial(find_device("DATA"))
d.flushInput()
v=d.read(4096*128)

In [365]:
%timeit d.read(20_000_000)

4.9 s ± 2.03 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
np.where(np.array(list(v), dtype=np.uint8)==0xff)

In [142]:
print(v[4080:4110])

b'\xed\x80\x00\x00\xee\x80\x00\x00\xef\x80\x00\x00\xff\x00\xff\x00\x00\xff\x00\xff\xc9f\x00\x00\xcaf\x00\x00\xcbf'


In [497]:
plt.figure()
data = np.diff(np.frombuffer(v, dtype=np.uint32))
data[data>1e7]=0
plt.plot(data)

In [560]:
values = np.frombuffer(v, dtype=np.int16).copy()

In [392]:
plt.figure()
plt.plot(np.gradient(values))

In [628]:
values = np.frombuffer(v, dtype=np.int16).copy()[6000:]
for i in range(10):
    if values[i]==-3856:
        values = values[i:].copy()
plt.figure()
for i in range(2,6):
    plt.plot(values[i::6])

In [606]:
plt.figure()
plt.psd(values[3::6].copy(), Fs=2e5, detrend='mean', scale_by_freq=True, NFFT=4096)
#plt.psd(values[3::6].copy(), Fs=2e5, detrend='mean', scale_by_freq=True, NFFT=4096)
#plt.psd(values[4::6].copy(), Fs=2e5, detrend='mean', scale_by_freq=True, NFFT=4096)
#plt.psd(values[5::6].copy(), Fs=2e5, detrend='mean', scale_by_freq=True, NFFT=4096)

(array([4.33121807e-06, 2.37783529e-05, 3.86303005e-05, ...,
        5.80575366e-05, 8.68493859e-05, 3.77967837e-05]),
 array([0.00000000e+00, 4.88281250e+01, 9.76562500e+01, ...,
        9.99023437e+04, 9.99511719e+04, 1.00000000e+05]))

In [549]:

for i in range(len(v)):
    if v[i]==0xFF and v[i+1] == 0 and v[i+2]== 0xFF and v[i+3]==0:
        print(i)